# Strategy
## 1. Collect features:
    1.1 Service Orders SAPSR3./BIC/AZCS_O0100 (not working now)
    1.2 Equipment sap_bi.bic_azequip00_int
    1.3 Material smithd2.material_features
    1.4 BOM smithd2.material_bom_exploded_enterprise
## 2. Cluster relevant materials
    2.1 Create table of materials on service orders
    2.2 Create distance metric
    2.3 Cluster
## 3. Create TARGET: component pctReplaced feature (Qty replaced / Qty serviced)
    3.1 Get Qty of each top level material serviced (with something replaced)
    3.2 Get Qty of each component exposed to service
    3.3 Get Qty of each component replaced
    3.4 Generate Metric
## 4. Create master table of features and Target

In [2]:
%run '00_database_connectivity_setup.ipynb'

## 2. Cluster relevant materials
### 2.1 Create table of materials to be clustered

In [54]:
%%execsql
--get list of materials to be clustered
DROP TABLE if EXISTS nate.aro_components;
CREATE TABLE nate.aro_components AS 
SELECT distinct b.material
    from sap.service_order_int a
    inner join data_science.ordercomponents_int b
    on a.order_txt=b.ordr
DISTRIBUTED BY(material);

In [33]:
%%showsql
SELECT COUNT(*) from nate.aro_components;

,count
0,20804


In [27]:
%%showsql

SELECT COUNT(*) from 
( SELECT distinct a.material
    from sap.service_order_int a
    inner join data_science.ordercomponents_int b
    on a.order_txt=b.ordr
) a
left join nate.aro_components b
on a.material=b.material
where b.material is null

,count
0,2491


In [35]:
%%execsql
DROP TABLE if EXISTS nate.aro_materials;
CREATE TABLE nate.aro_materials AS 
 SELECT DISTINCT a.material 
    from sap.service_order_int a
    inner join data_science.ordercomponents_int b
    on a.order_txt=b.ordr    
DISTRIBUTED BY (material);

In [3]:
%%showsql

select count(distinct material) from sudip.material_features_distance_components

,count
0,17806


In [4]:
%%showsql

select count(*) from nate.hclust_largest_results;

,count
0,7530


#### NOTE: About 2500 of the materials on the orders are NOT in the cluster.

CLUSTER THEM ALONE! there are only 7000 of them!

### 2.2 Create Distance metric

In [56]:
%%execsql
--create distance metrics
DROP TABLE IF EXISTS nate.material_features_distance_components;
set statement_timeout = 0; 
CREATE TABLE nate.material_features_distance_components as
(    
    SELECT a.material as material, b.material as similar_material,     
     CASE WHEN a.material=b.material then 0
    --this case whenspeeds it up by limiting heavy math to at least one key field match
    WHEN
       ( a.prod_line = b.prod_line or a.model=b.model or  a.commodity_code=b.commodity_code
            or a.prod_family=b.prod_family or a.material=b.based_on or a.based_on=a.material or a.desc1=b.desc1) 
     THEN            
       ( case when a.weight>0.1 and b.weight>0.1 then 0.3 * (abs(a.weight-b.weight)/GREATEST(a.weight,b.weight)) else 0.1 end +    
        CASE WHEN a.model=b.model then 0 else 0.3 end +    
        case when a.material=b.based_on or a.based_on=a.material then 0 else 0.3 end +        
        (1.0 - smithd2.jaccard_index(a.documents, b.documents)) * .3 +   
        (1.0 - smithd2.jaccard_index(a.components, b.components)) * .4 + 
        case when char_length(a.desc1)>0 and char_length(b.desc1)>0 
            then 0.3 * pdltools.demerau_levenshtein_distance( a.desc1, b.desc1 )
                 / GREATEST(char_length(a.desc1),char_length(b.desc1))  
            else 0 end +    
        case when char_length(a.desc2)>0 and char_length(b.desc2)>0 
            then 0.2 * pdltools.demerau_levenshtein_distance( a.desc2, b.desc2 )
                 / GREATEST(char_length(a.desc2),char_length(b.desc2))  
            else 0 end + 
        case when char_length(a.desc3)>0 and char_length(b.desc3)>0 
            then 0.2 * pdltools.demerau_levenshtein_distance( a.desc3, b.desc3 )
                 / GREATEST(char_length(a.desc3),char_length(b.desc3))  
           else 0 end +
        case when char_length(a.desc4)>0 and char_length(b.desc4)>0 
           then 0.2 * pdltools.demerau_levenshtein_distance( a.desc4, b.desc4 )
                 / GREATEST(char_length(a.desc4),char_length(b.desc4))  
            else 0 end + 
        case when a.uom=b.uom then 0 else 0.3 end +
        case when a.prod_family=b.prod_family then 0 else 0.1 end +
        case when a.prod_line=b.prod_line then 0 else 0.1 end +
        (1.0 - smithd2.jaccard_index(a.matlspecs, b.matlspecs)) * .2 +
        (1.0 - smithd2.jaccard_index(a.weldspecs, b.weldspecs)) * .2 +
        case when a.serialization=b.serialization 
            or a.serialization is null and b.serialization is null then 0 
            when a.serialization is not null and b.serialization is not null         
            then 0.05 else 0.2 end +     
        (1.0 - smithd2.jaccard_index(a.qspecs, b.qspecs)) * .2 +
        (1.0 - smithd2.jaccard_index(a.qmsects, b.qmsects)) * .2 +       
        (1.0 - smithd2.jaccard_index(a.coatingspecs, b.coatingspecs)) * .2 +           
        case when a.material_type=b.material_type then 0 else 0.2 end +    
        case when a.commodity_code=b.commodity_code then 0 else 0.1 end) / 4.2
      ELSE 1.0 END
        as score
    FROM smithd2.material_features a
    INNER JOIN nate.aro_components c
    ON a.material = c.material
    CROSS JOIN smithd2.material_features b
    INNER JOIN  nate.aro_components d
    ON b.material = d.material
    where 
        a.material>=b.material  
)
DISTRIBUTED BY (material);

DatabaseError: Execution failed on sql '--create distance metrics
DROP TABLE IF EXISTS nate.material_features_distance_components;
set statement_timeout = 0; 
CREATE TABLE nate.material_features_distance_components as
(    
    SELECT a.material as material, b.material as similar_material,     
     CASE WHEN a.material=b.material then 0
    --this case whenspeeds it up by limiting heavy math to at least one key field match
    WHEN
       ( a.prod_line = b.prod_line or a.model=b.model or  a.commodity_code=b.commodity_code
            or a.prod_family=b.prod_family or a.material=b.based_on or a.based_on=a.material or a.desc1=b.desc1) 
     THEN            
       ( case when a.weight>0.1 and b.weight>0.1 then 0.3 * (abs(a.weight-b.weight)/GREATEST(a.weight,b.weight)) else 0.1 end +    
        CASE WHEN a.model=b.model then 0 else 0.3 end +    
        case when a.material=b.based_on or a.based_on=a.material then 0 else 0.3 end +        
        (1.0 - smithd2.jaccard_index(a.documents

In [37]:
%%execsql
--create distance metrics
DROP TABLE IF EXISTS nate.material_features_distance_materials;
set statement_timeout = 0; 
CREATE TABLE nate.material_features_distance_materials as
(    
    SELECT a.material as material, b.material as similar_material,     
     CASE WHEN a.material=b.material then 0
    --this case whenspeeds it up by limiting heavy math to at least one key field match
    WHEN
       ( a.prod_line = b.prod_line or a.model=b.model or  a.commodity_code=b.commodity_code
            or a.prod_family=b.prod_family or a.material=b.based_on or a.based_on=a.material or a.desc1=b.desc1) 
     THEN            
       ( case when a.weight>0.1 and b.weight>0.1 then 0.3 * (abs(a.weight-b.weight)/GREATEST(a.weight,b.weight)) else 0.1 end +    
        CASE WHEN a.model=b.model then 0 else 0.3 end +    
        case when a.material=b.based_on or a.based_on=a.material then 0 else 0.3 end +        
        (1.0 - smithd2.jaccard_index(a.documents, b.documents)) * .3 +   
        (1.0 - smithd2.jaccard_index(a.components, b.components)) * .4 + 
        case when char_length(a.desc1)>0 and char_length(b.desc1)>0 
            then 0.3 * pdltools.demerau_levenshtein_distance( a.desc1, b.desc1 )
                 / GREATEST(char_length(a.desc1),char_length(b.desc1))  
            else 0 end +    
        case when char_length(a.desc2)>0 and char_length(b.desc2)>0 
            then 0.2 * pdltools.demerau_levenshtein_distance( a.desc2, b.desc2 )
                 / GREATEST(char_length(a.desc2),char_length(b.desc2))  
            else 0 end + 
        case when char_length(a.desc3)>0 and char_length(b.desc3)>0 
            then 0.2 * pdltools.demerau_levenshtein_distance( a.desc3, b.desc3 )
                 / GREATEST(char_length(a.desc3),char_length(b.desc3))  
           else 0 end +
        case when char_length(a.desc4)>0 and char_length(b.desc4)>0 
           then 0.2 * pdltools.demerau_levenshtein_distance( a.desc4, b.desc4 )
                 / GREATEST(char_length(a.desc4),char_length(b.desc4))  
            else 0 end + 
        case when a.uom=b.uom then 0 else 0.3 end +
        case when a.prod_family=b.prod_family then 0 else 0.1 end +
        case when a.prod_line=b.prod_line then 0 else 0.1 end +
        (1.0 - smithd2.jaccard_index(a.matlspecs, b.matlspecs)) * .2 +
        (1.0 - smithd2.jaccard_index(a.weldspecs, b.weldspecs)) * .2 +
        case when a.serialization=b.serialization 
            or a.serialization is null and b.serialization is null then 0 
            when a.serialization is not null and b.serialization is not null         
            then 0.05 else 0.2 end +     
        (1.0 - smithd2.jaccard_index(a.qspecs, b.qspecs)) * .2 +
        (1.0 - smithd2.jaccard_index(a.qmsects, b.qmsects)) * .2 +       
        (1.0 - smithd2.jaccard_index(a.coatingspecs, b.coatingspecs)) * .2 +           
        case when a.material_type=b.material_type then 0 else 0.2 end +    
        case when a.commodity_code=b.commodity_code then 0 else 0.1 end) / 4.2
      ELSE 1.0 END
        as score
    FROM smithd2.material_features a
    INNER JOIN nate.aro_materials c
    ON a.material = c.material
    CROSS JOIN smithd2.material_features b
    INNER JOIN  nate.aro_materials d
    ON b.material = d.material
    where 
        a.material>=b.material  
)
DISTRIBUTED BY (material);

In [42]:
%%showsql

select count(*) from sudip.material_features_distance_components;

,count
0,158535721


### 2.3 Create Clusters

In [ ]:
# actually large, ran with 0.75
sql = """
drop table if exists nate.hclust_largest_results_components;
create table nate.hclust_largest_results_components
as
(
    select (result).material, (result).cluster_number
    from
    (
        select nate.run_hclust(key, 0.75) as result 
        from 
        (
            select
                nate.stack_rows( ARRAY['material', 'similar_material', 'score'],
                    material, 
                    similar_material, 
                    score) as key
            from sudip.material_features_distance_components
            where  material >= similar_material              
        ) q1
    ) q2
) distributed by (material);
"""
    
psql.execute(sql, conn)
conn.commit()

In [45]:
# actually large, ran with 0.75
sql = """
drop table if exists nate.hclust_largest_results_materials;
create table nate.hclust_largest_results_materials
as
(
    select (result).material, (result).cluster_number
    from
    (
        select nate.run_hclust(key, 0.75) as result 
        from 
        (
            select
                nate.stack_rows( ARRAY['material', 'similar_material', 'score'],
                    material, 
                    similar_material, 
                    score) as key
            from nate.material_features_distance_materials
            where  material >= similar_material              
        ) q1
    ) q2
) distributed by (material);
"""
    
psql.execute(sql, conn)
conn.commit()

In [49]:
# actually large, ran with 0.85
sql = """
drop table if exists nate.hclust_materials85;
create table nate.hclust_materials85
as
(
    select (result).material, (result).cluster_number
    from
    (
        select nate.run_hclust(key, 0.85) as result 
        from 
        (
            select
                nate.stack_rows( ARRAY['material', 'similar_material', 'score'],
                    material, 
                    similar_material, 
                    score) as key
            from nate.material_features_distance_materials
            where  material >= similar_material              
        ) q1
    ) q2
) distributed by (material);
"""
    
psql.execute(sql, conn)
conn.commit()

In [52]:
%%showsql
SELECT max(cluster_number) from nate.hclust_materials85
limit 5;

,max
0,80


## 3. Create TARGET: component pctReplaced feature (Qty replaced / Qty serviced)

In [21]:
%%execsql

--3.1 Get Qty of each top level material serviced (with something replaced)
DROP TABLE IF EXISTS nate.serviced_materials;
CREATE TABLE nate.serviced_materials AS 
(
    SELECT a.material, plant, count(distinct order_txt) as order_qty
    FROM sap.service_order_int a
    inner join data_science.ordercomponents_int b
    on a.order_txt = b.ordr
    group by 1, 2
) DISTRIBUTED BY (material);

In [22]:
%%execsql
--3.2 Get Qty of each component exposed to service
DROP TABLE IF EXISTS nate.serviced_components;
CREATE TABLE nate.serviced_components AS 
(
    select c.material, c.plant, d.component, d.unit as uom, sum(c.order_qty*d.quantity) as qty_serviced
    from nate.serviced_materials c
    inner join smithd2.material_bom_exploded_enterprise d
    on c.material=d.material
    group by 1, 2, 3, 4
) DISTRIBUTED BY (component);


In [23]:
%%execsql
--3.3 Get Qty of each component replaced
DROP TABLE IF EXISTS nate.replaced_components;
CREATE TABLE nate.replaced_components AS 
( 
    SELECT h.material, plant, g.material as component, g.uom, sum(g.qtywithdrawn) as qty_replaced
    from data_science.ordercomponents_int g
    inner join sap.service_order_int h 
    on g.ordr = h.order_txt
    group by 1,2,3,4
) DISTRIBUTED BY (component);


In [24]:
%%execsql
--3.4 Generate Metric
DROP TABLE IF EXISTS nate.components_pct_replaced;
CREATE TABLE nate.components_pct_replaced AS 
(
    SELECT e.material, e.plant, e.component, e.uom, 
            case when f.qty_replaced is null then 0 else f.qty_replaced end as qty_replaced,
            e.qty_serviced,
            case when f.qty_replaced is null or e.qty_serviced=0 then 0 
                else f.qty_replaced / e.qty_serviced end as pctReplaced 
    from  nate.serviced_components e
    left outer join nate.replaced_components f
    on e.material=f.material and e.plant=f.plant and e.component=f.component and e.uom=f.uom
) DISTRIBUTED BY (component);

In [25]:
%%showsql
select * from  nate.components_pct_replaced
limit 100;

,material,plant,component,uom,qty_replaced,qty_serviced,pctreplaced
0,100007548,KOS2,100031210,EA,0.0,4.0,0.000000
1,100009270,EWAA,100001136,EA,0.0,1.0,0.000000
2,100009270,EWAA,200004341,EA,0.0,8.0,0.000000
3,100009270,EWAA,200012724,EA,0.0,13.0,0.000000
4,100009270,EWAA,7104909,EA,0.0,13.0,0.000000
5,100009270,KOS2,100001136,EA,0.0,7.0,0.000000
6,100009270,KOS2,200004341,EA,8.0,56.0,0.142857
7,100009270,KOS2,200012724,EA,36.0,91.0,0.395604
8,100009270,KOS2,7104909,EA,0.0,91.0,0.000000
9,100026839,KOS2,100014923,EA,0.0,20.0,0.000000


## 4. Create master table of features and Target

In [91]:
%%execsql
DROP TABLE IF EXISTS nate.component_features;
CREATE TABLE nate.component_features 
AS 
(
    SELECT a. material, a,plant, a.component, a.uom, a.qty_replaced, a.qty_serviced, a.pctreplaced,   
    
    b.weight as material_weight,
    b.material_type as material_material_type,  
    b.material_group,
    
    b.surface_matl as material_surface_matl, 
    b.subsea_matl as material_subsea_matl,     
    (b.material_type is not null) as has_materialtype,
    (b.coatingspecs is not null) as has_coatings,
    (b.documents is not null) as has_documents, 
    (b.matlspecs is not null) as has_matlspecs,
    (b.weldspecs is not null) as has_weldspecs, 
    (b.qspecs is not null) as has_qspecs    
   
    FROM  nate.components_pct_replaced a
    INNER JOIN smithd2.material_features b  ON a.component = b.material     
    --INNER join nate.cluster_master c on a.component=c.material
) DISTRIBUTED BY (component);


In [89]:
%%showsql 
SELECT * FROM  nate.component_features;

,component,uom,qty_replaced,qty_serviced,pctreplaced,material_weight,material_material_type,material_group,material_surface_matl,material_subsea_matl,has_materialtype,has_coatings,has_documents,has_matlspecs,has_weldspecs,has_qspecs
0,P2000034376,EA,0.00,4.0,0.000000,17.000,HALB,M-M04-000,False,True,True,False,True,False,False,False
1,P1000075598,EA,0.00,102.0,0.000000,1784.000,HALB,O-C04-000,False,True,True,False,True,False,False,False
2,180-583-015,PCE,0.00,20.0,0.000000,0.000,HALB,99,True,True,True,False,True,False,False,False
3,P6000062076,EA,0.00,30.0,0.000000,9.900,HALB,A-T03-ROV,False,True,True,False,True,False,False,False
4,P1000086127,EA,0.00,19.0,0.000000,174.000,HALB,F-S01-MMW,True,True,True,True,False,True,False,False
5,P156290,EA,2.00,20.0,0.100000,0.000,HALB,O-C05-000,True,True,True,False,True,False,False,False
6,P1000011123,EA,8.00,28.0,0.285714,5.000,HALB,O-V04-NE0,True,True,True,False,False,False,False,False
7,701-072-387,EA,270.00,168.0,1.607143,0.300,HALB,O-F03-000,True,True,True,True,False,True,False,False
8,P6000063105,EA,10.00,38.0,0.263158,5338.000,HALB,F-S01-LLA,False,True,True,False,True,False,False,False
9,708-610-003,PCE,237.00,156039.0,0.001519,0.000,HALB,99,True,True,True,False,True,False,False,False


# SCRATCH

In [84]:
%%showsql
select * 
from sap.service_order_int
where equipment is not null
limit 10

,order_txt,material,system_status,created_on,release_txt,schedstart,sched_fin,act_start,actual_end,profit_ctr,plnt,plpl,pg,type,notifctn,equipment
0,1460720,51-094-046,CLSD PCNF PRT MANC NMAT PRC SETC,6/13/2016,6/20/2016,6/13/2016,6/13/2016,6/23/2016,None,P9321028,EWBV,EWBV,200,XP01,None,10079657
1,1398806,P2000026290,TECO PCNF GMPS MANC PPRT PRC SETC,7/23/2015,7/23/2015,7/24/2015,7/28/2015,11/16/2015,None,P9321028,EWBV,EWBV,200,XP01,None,12824348
2,1321000,P504868,CLSD PCNF PRT MANC PRC SETC,7/2/2014,7/2/2014,7/14/2014,7/14/2014,7/9/2014,None,P9321028,EWBV,EWBV,200,XP01,None,13273592
3,1426835,DP-18-5198,TECO PCNF PRT MANC NMAT PRC SETC,12/15/2015,12/15/2015,12/15/2015,12/15/2015,12/17/2015,None,P9321028,EWBV,EWBV,200,XP01,None,13787729
4,1368158,P1000057548,TECO PCNF GMPS MANC PPRT PRC SETC,3/4/2015,3/4/2015,3/26/2015,4/17/2015,3/24/2015,None,P9321028,EWBV,EWBV,200,XP01,None,13839525
5,1318159,P1000057548,CLSD PCNF PRT GMPS MANC PRC SETC,6/19/2014,6/19/2014,6/20/2014,6/24/2014,6/23/2014,None,P9321028,EWBV,EWBV,200,XP01,None,13839525
6,1335525,P1000057548,CLSD PCNF GMPS MANC PPRT PRC SETC,9/17/2014,9/17/2014,9/17/2014,9/19/2014,9/17/2014,None,P9321028,EWBV,EWBV,200,XP01,None,13839525
7,1341098,P1000039291,CLSD PCNF PRT MANC PRC SETC,10/16/2014,10/16/2014,10/16/2014,10/16/2014,10/16/2014,None,P9321028,EWBV,EWBV,200,XP01,None,13841165
8,1382663,P1000039291,TECO PCNF PRT MANC PRC SETC,5/11/2015,5/11/2015,5/11/2015,5/13/2015,9/10/2015,None,P9321028,EWBV,EWBV,200,XP01,None,13841165
9,1377611,P2000040547,TECO CNF PRT MANC PRC SETC,4/17/2015,4/17/2015,4/20/2015,4/20/2015,4/20/2015,5/13/2015,P9321028,EWBV,EWBV,200,XP01,None,13841168


In [ ]:
%%execsql
-- this table has 1 row for each component replaced on a service order, plus other features

DROP TABLE IF EXISTS nate.components_pct_replaced_features;
CREATE TABLE nate.components_pct_replaced_features AS 
    (
    SELECT a.ordr, 
        case when asong.is_numeric(c.material) then trim(leading '0' from c.material) else c.material end as material
        , e.txtmd as material_desc, a.material AS component, d.txtmd as component_desc, a.qtywithdrawn, a.uom, a.plant
    FROM data_science.ordercomponents_int a 
    JOIN sap_bi.bic_azcs_o0100_int b
    ON a.ordr = trim(leading '0' from b.cs_order)
    JOIN sap_bi.bic_azequip00_int c
    ON c.equipment = b.equipment
    JOIN sap_bi.bi0_tmaterial_int d
    ON d.material = a.material
    JOIN sap_bi.bi0_tmaterial_int e
    ON e.material = c.material
    WHERE a.qtywithdrawn > 0 and a.material is not null and char_length(a.material) > 0 and d.langu = 'E' and e.langu = 'E'
    )
DISTRIBUTED BY (ordr);

In [60]:
%%execsql
DROP TABLE IF EXISTS nate.component_features;
CREATE TABLE nate.component_features 
AS (SELECT a.component, a.orders_replaced, a.orders_serviced, a.pctreplaced,    
    
    b.weight as material_weight,
    b.material_type as material_material_type,  
    b.material_group,
    b.surface_matl as material_surface_matl, 
    b.subsea_matl as material_subsea_matl, 
    
    (b.material_type is not null) as has_materialtype,
    (b.coatingspecs is not null) as has_coatings,
    (b.documents is not null) as has_documents, 
    (b.matlspecs is not null) as has_matlspecs,
    (b.weldspecs is not null) as has_weldspecs, 
    (b.qspecs is not null) as has_qspecs,    
    c.*
    FROM  nate.replacedvsserviced a
    INNER JOIN smithd2.material_features b  ON a.component = b.material 
    INNER join nate.cluster_master c on a.component=c.material
    )

In [ ]:
%%execsql
DROP TABLE IF EXISTS nate.full_features;
CREATE TABLE nate.full_features 
AS (SELECT a.ordr, a.material, a.material_desc, a.component, 
    a.component_desc, a.uom, a.plant, b.orders_replaced, b.orders_serviced, b.pctreplaced,
    c.ord_proccd as leadtimecalday, c.ord_procwd as leadtimeworkday, c.calday, 
    c.pmplangrp, c.pmacttype, c.division, c.customer, c.funct_loc, c.salesorg, 
    c.spl_refdt as refdat, c.bic_zabckz as ABCinc, c.ord_intime, c.aedat as changedon, 
    c.bic_zaenam as changedby, c.bic_zactfinsh as actfinish, c.actstartdt as actstart,
    c.bic_zactfinsh, c.bic_zreledt, c.bic_zakknz as category, c.bic_zawerk, 
    c.bic_zcrtdat, c.bic_zsystatus, c.workcenter, c.bus_area, c.bic_zphase, c.costcenter, c.priority, 
    c.bic_zernam as createdby, d.coatingspecs as material_coatingspecs, d.documents as material_documents, 
    d.matlspecs as material_matlspecs, d.weldspecs as material_weldspecs, 
    d.qspecs as material_qspecs, d.weight as material_weight, d.model as material_model, 
    d.material_type as material_material_type, 
    d.material_group as material_material_group, d.surface_matl as material_surface_matl, 
    d.subsea_matl as material_subsea_matl, e.coatingspecs as component_coatingspecs, 
    e.documents as component_documents, e.matlspecs as component_matlspecs, e.weldspecs 
    as component_weldspecs, e.qspecs as component_qspecs, e.weight as component_weight, 
    e.model as component_model, e.material_type as component_material_type, e.material_group as component_material_group, 
    e.surface_matl as component_surface_matl,e.subsea_matl as component_subsea_matl
FROM nate.simple_features a
JOIN nate.replacedvsserviced b
ON a.material = b.component
JOIN sap_bi.bic_azcs_o0100_int c
ON a.ordr = trim(leading '0' from c.cs_order)
JOIN smithd2.material_features d
ON a.material = d.material 
JOIN smithd2.material_features e
ON a.component = e.material)

In [ ]:
%%execsql
DROP TABLE IF EXISTS nate.prediction_input;
CREATE TABLE nate.prediction_input AS
SELECT component, material, orders_replaced, orders_serviced, pctreplaced, leadtimecalday, 
my_to_date(actstart, 'YYYYMMDD') as actstart, ABCinc, bic_zphase::int, (material_coatingspecs is not null) as matl_has_coatings,
(material_documents is not null) as matl_has_documents, (material_matlspecs is not null) 
as matl_has_matlspecs,
(material_weldspecs is not null) as matl_has_weldspecs, (material_qspecs is not null) 
as matl_has_qspecs, (component_coatingspecs is not null) as has_coatings,
(component_documents is not null) as has_documents, (component_matlspecs is not null) 
as has_matlspecs,
(component_weldspecs is not null) as has_weldspecs, (component_qspecs is not null) 
as has_qspecs, 
material_weight, material_surface_matl, material_subsea_matl, component_weight, component_surface_matl, 
component_subsea_matl,
b.*
FROM nate.full_features a
inner join nate.cluster_master b on a.component=b.material